In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
df = pd.read_excel ('sales dataset.xlsx', dtype={'InvoiceNo': str, 'Description': str})
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


**Selecting UK for analysis**

In [3]:
df = df[df['Country'] == 'United Kingdom']
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom


**Checking the shape of dataset**

In [4]:
df.shape

(495478, 8)

**Checking for missing values**

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

**Selecting relevant columns**

In [6]:
df = df[['InvoiceNo', 'Description', 'Quantity']]
df.head(1)

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6


**Checking for missing values again since we are now working with just these settings**

In [7]:
df.isnull().sum()

InvoiceNo         0
Description    1454
Quantity          0
dtype: int64

**Dropping missing values**

In [8]:
df = df.dropna()

**Checking for missing values again**

In [9]:
df.isnull().sum()

InvoiceNo      0
Description    0
Quantity       0
dtype: int64

**Checking the data types of columns**

In [10]:
df.dtypes

InvoiceNo      object
Description    object
Quantity        int64
dtype: object

**Removing empty spaces from columns**

In [11]:
df['InvoiceNo'] = df['InvoiceNo'].str.strip()
df['Description'] = df['Description'].str.strip()

**Removing cash transactions from the dataset**

In [12]:
df = df[~df['InvoiceNo'].str.contains('C')]
df.head()

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6


**Converting the dataset in necessary format for fpgrowth algorithm**

In [13]:
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [14]:
basket.head(3)

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Converting the data to boolean format for FP-Growth**

In [15]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units).astype(bool)
basket_sets.head(1)

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**In the context of market basket analysis, the minimum support is a parameter that is used to set a threshold for the minimum frequency of occurrence of an itemset in the dataset. A minimum support of 0.1 means that we are interested in finding itemsets that appear in at least 10% of the transactions in the dataset.**

**Looking for items that are frequently bought together with a min support of 9%**

In [16]:
frequent_itemsets = fpgrowth(basket_sets, min_support=0.009, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.116027,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.023998,(KNITTED UNION FLAG HOT WATER BOTTLE)
2,0.022016,(RED WOOLLY HOTTIE WHITE HEART.)
3,0.018213,(SET 7 BABUSHKA NESTING BOXES)
4,0.015856,(WHITE METAL LANTERN)
...,...,...
2601,0.014142,"(HOT WATER BOTTLE KEEP CALM, CHOCOLATE HOT WAT..."
2602,0.009214,"(HOT WATER BOTTLE KEEP CALM, GARDENERS KNEELIN..."
2603,0.010178,"(HAND WARMER UNION JACK, HAND WARMER RED LOVE ..."
2604,0.011517,"(HAND WARMER OWL DESIGN, HAND WARMER RED LOVE ..."


**In Market basket analysis, the minimum lift is another threshold parameter that helps to filter out uninteresting or weak rules. The lift measures the strength of association between two items or itemsets, and is defined as the ratio of the observed frequency of co-occurrence of those items to the frequency that would be expected if they were independent. A minimum lift value is set to only return the rules that have a lift value greater than or equal to this threshold. This helps to filter out rules that may have a high support value but may not be useful or interesting as they do not have a strong enough association between the items or itemsets. For example, if the minimum lift is set to 1.1, only rules with a lift value of 1.1 or greater will be returned. A lift value less than 1 indicates negative correlation, 1 indicates independence, and greater than 1 indicates positive correlation. Therefore, a minimum lift of 1.1 means that only rules with a positive correlation stronger than the correlation between two independent items will be returned.**

In [17]:
# Generating association rules using minimum confidence, minimum lift, and minimum length.
# In online retail mostly 0.5 threshold of confidence is taken
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)
rules = rules[(rules['lift'] >= 1.1) & (rules['consequents'].apply(lambda x: len(x)) >= 2) & (rules['antecedents'].apply(lambda x: len(x)) == 1)]

In [18]:
# Sort the rules by lift in descending order
rules = rules.sort_values(by=['lift'], ascending=False)

In [19]:
rules.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
810,(HERB MARKER CHIVES),"(HERB MARKER THYME, HERB MARKER PARSLEY, HERB ...",0.010874,0.010553,0.009267,0.852217,80.757270,0.009152,6.695259,0.998475
808,(HERB MARKER PARSLEY),"(HERB MARKER THYME, HERB MARKER ROSEMARY, HERB...",0.012213,0.009481,0.009267,0.758772,80.026861,0.009151,4.106150,0.999714
772,(HERB MARKER CHIVES),"(HERB MARKER PARSLEY, HERB MARKER MINT)",0.010874,0.010981,0.009481,0.871921,79.400120,0.009362,7.721953,0.998261
807,(HERB MARKER THYME),"(HERB MARKER PARSLEY, HERB MARKER ROSEMARY, HE...",0.012321,0.009481,0.009267,0.752174,79.330975,0.009150,3.996829,0.999712
765,(HERB MARKER PARSLEY),"(HERB MARKER THYME, HERB MARKER CHIVES)",0.012213,0.009856,0.009535,0.780702,79.207285,0.009415,4.515055,0.999583
...,...,...,...,...,...,...,...,...,...,...
1424,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.037658,0.037551,0.026409,0.701280,18.675463,0.024995,3.221913,0.983490
1425,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",0.050032,0.029248,0.026409,0.527837,18.047007,0.024945,2.055969,0.994338
1333,(ALARM CLOCK BAKELIKE CHOCOLATE),"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",0.020356,0.030159,0.010660,0.523684,17.364364,0.010046,2.036131,0.961993
1423,(ROSES REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",0.051264,0.030909,0.026409,0.515152,16.666982,0.024824,1.998751,0.990793


**Final results with only relevant columns**

In [20]:
rules_shrunk = rules[['antecedents', 'consequents', 'confidence', 'lift']]
rules_shrunk.head(100)

,antecedents,consequents,confidence,lift
810,(HERB MARKER CHIVES),"(HERB MARKER THYME, HERB MARKER PARSLEY, HERB ...",0.852217,80.757270
808,(HERB MARKER PARSLEY),"(HERB MARKER THYME, HERB MARKER ROSEMARY, HERB...",0.758772,80.026861
772,(HERB MARKER CHIVES),"(HERB MARKER PARSLEY, HERB MARKER MINT)",0.871921,79.400120
807,(HERB MARKER THYME),"(HERB MARKER PARSLEY, HERB MARKER ROSEMARY, HE...",0.752174,79.330975
765,(HERB MARKER PARSLEY),"(HERB MARKER THYME, HERB MARKER CHIVES)",0.780702,79.207285
...,...,...,...,...
1424,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.701280,18.675463
1425,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",0.527837,18.047007
1333,(ALARM CLOCK BAKELIKE CHOCOLATE),"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",0.523684,17.364364
1423,(ROSES REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",0.515152,16.666982
